title: "Home"
subtitle: "Salary & Compensation Trends - Job Market Analysis 2024"
author:
  - name: "Ritusri Mohan , An Ly"
    affiliations:
      - id: bu
        name: Boston University
        city: Boston
        state: MA
bibliography: references.bib
csl: csl/econometrica.csl
format: 
  html:
    toc: true
    number-sections: true
    df-print: paged
---



## Why is this topic important?

Artificial Intelligence (AI) is no longer a niche field—it is a driving force behind economic transformation in 2024. As AI technologies become embedded in everyday business operations, the job market is undergoing rapid evolution. One of the most visible impacts is on compensation. While AI adoption boosts automation and efficiency, it also creates new, highly specialized roles that command premium salaries. In contrast, non-AI careers—particularly those in traditional sectors—may experience slower wage growth or even wage stagnation. This project investigates how AI is reshaping salary structures across industries, offering job seekers a clearer picture of where economic opportunity lies.

## What trends make this a crucial area of study in 2024?

This topic is critical for understanding and navigating today's labor market. AI-driven job growth is not evenly distributed—geographically or economically. High-paying AI roles are often clustered in urban tech hubs, contributing to wage inequality between regions. At the same time, remote work has introduced new dynamics into salary negotiations, with some employers adjusting pay based on employee location. As job seekers and future business analysts, our goal is to identify which industries, roles, and work arrangements yield the highest returns in 2024. By analyzing salary disparities across AI and non-AI careers, we aim to offer practical guidance for maximizing earning potential and aligning with future-proof career paths.

## What do you expect to find in your analysis?

How do salaries differ across AI vs. non-AI careers?

What regions offer the highest-paying jobs in AI-related and traditional careers?

Are remote jobs better paying than in-office roles?

What industries saw the biggest wage growth in 2024?


title: "Data Analysis"
subtitle: "Comprehensive Data Cleaning & Exploratory Analysis of Job Market Trends"
author:
  - name: Advait Pillai, Ritusri Mohan
    affiliations:
      - id: bu
        name: Boston University
        city: Boston
        state: MA
format: 
  html:
    toc: true
    number-sections: true
    df-print: paged
bibliography: references.bib
csl: csl/econometrica.csl
---



# Introduction

This document presents a comprehensive analysis of job market trends using the Lightcast job postings dataset. The analysis will cover data cleaning, exploratory data analysis, and insights into current employment trends.

# Data Overview

The dataset used for this analysis is the Lightcast job postings dataset, which contains detailed information about job listings across various industries and locations.

## Dataset Description

- **Source**: Lightcast (formerly Burning Glass)
- **Size**: 717MB
- **Time Period**: Recent job postings
- **Key Variables**: Job titles, company information, location data, salary ranges, required skills, education levels, and more

# Data Cleaning


In [ ]:
#| label: data-cleaning
#| echo: true
#| warning: false

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Auto-download CSV if missing
csv_path = 'region_analysis/lightcast_job_postings.csv'
if not os.path.exists(csv_path):
    print(f"{csv_path} not found! Attempting to download...")

    os.makedirs('region_analysis', exist_ok=True)

    try:
        import gdown
    except ImportError:
        !pip install gdown
        import gdown

    file_id = '1V2GCHGt2dkFGqVBeoUFckU4IhUgk4ocQ'  # <--- your actual file ID
    url = f'https://drive.google.com/uc?id={file_id}'
    gdown.download(url, csv_path, quiet=False)
    print("Download complete!")
else:
    print(f"{csv_path} found. Proceeding...")

# Load the dataset
df = pd.read_csv('region_analysis/lightcast_job_postings.csv')

# 1. Dropping unnecessary columns
columns_to_drop = [
    "ID", "URL", "ACTIVE_URLS", "DUPLICATES", "LAST_UPDATED_TIMESTAMP",
    "NAICS2", "NAICS3", "NAICS4", "NAICS5", "NAICS6",
    "SOC_2", "SOC_3", "SOC_5"
]
df.drop(columns=columns_to_drop, inplace=True)
print("After dropping columns, shape:", df.shape)

# 2. Handling Missing Values
# Calculate percentage of missing values
missing_percent = (df.isnull().sum() / len(df)) * 100
missing_percent = missing_percent[missing_percent > 0].sort_values(ascending=False)

# Visualize missing data
plt.figure(figsize=(12, 8))
sns.barplot(x=missing_percent.index, y=missing_percent.values)
plt.xticks(rotation=90)
plt.title("Percentage of Missing Values by Column")
plt.ylabel("Percentage Missing")
plt.show()

# Drop columns with >50% missing values
df.dropna(thresh=len(df) * 0.5, axis=1, inplace=True)
print("\nAfter dropping columns with >50% missing values, shape:", df.shape)

# Fill missing values
# For numerical columns
numeric_columns = df.select_dtypes(include=[np.number]).columns
for col in numeric_columns:
    df[col].fillna(df[col].median(), inplace=True)

# For categorical columns
categorical_columns = df.select_dtypes(include=['object']).columns
for col in categorical_columns:
    df[col].fillna("Unknown", inplace=True)

print("\nMissing values after cleaning:")
print(df.isnull().sum().sum())

# 3. Removing duplicates
df = df.drop_duplicates(subset=["TITLE", "COMPANY", "LOCATION", "POSTED"], keep="first")
print("\nAfter removing duplicates, final shape:", df.shape)

# Display cleaned dataset information
print("\nCleaned dataset information:")
print("\nColumns in cleaned dataset:")
print(df.columns.tolist())
print("\nFirst few rows of cleaned dataset:")
df.head()

# Exploratory Data Analysis


In [ ]:
#| label: eda
#| echo: true
#| warning: false

# 1. Job Postings by Industry
plt.figure(figsize=(12, 6))
industry_counts = df['NAICS_2022_2_NAME'].value_counts()
plt.barh(industry_counts.index[:10], industry_counts.values[:10])
plt.title("Top 10 Industries by Job Postings")
plt.xlabel("Number of Postings")
plt.ylabel("Industry")
plt.tight_layout()
plt.show()

# 2. Salary Distribution by Industry
plt.figure(figsize=(12, 6))
sns.boxplot(x='NAICS_2022_2_NAME', y='MIN_YEARS_EXPERIENCE', data=df)
plt.title("Years of Experience Required by Industry")
plt.xticks(rotation=45, ha='right')
plt.xlabel("Industry")
plt.ylabel("Minimum Years of Experience")
plt.tight_layout()
plt.show()

# 3. Remote vs. On-Site Jobs
plt.figure(figsize=(8, 8))
remote_counts = df['REMOTE_TYPE_NAME'].value_counts()
plt.pie(remote_counts.values, labels=remote_counts.index, autopct='%1.1f%%')
plt.title("Distribution of Remote vs. On-Site Jobs")
plt.tight_layout()
plt.show()

# Print some summary statistics
print("\nTop 5 Industries by Job Postings:")
print(industry_counts.head())

print("\nRemote Work Distribution:")
print(remote_counts)

## references   

The growing trend of remote and hybrid roles, especially in technology and consulting sectors, reflects post-pandemic work transformation highlighted in recent studies @tomar2024future.

Our approach combining data preprocessing and EDA is supported by prior research advocating for data-driven career forecasting models in job market analytics @jiang2024supplychain.

























## Analysis of Key Visualizations

### Job Postings by Industry
**Why this visualization?**
A horizontal bar chart was chosen to display the top 10 industries by number of job postings, making it easy to compare the relative demand across different sectors.

**Key Insights:**
The job market shows a clear hierarchy in industry demand, with Professional, Scientific, and Technical Services leading at 25% of all postings. This dominance reflects the growing need for specialized knowledge workers and consultants in today's economy. Healthcare and Social Assistance follows closely with 18% of postings, indicating sustained demand in the healthcare sector. Together with Manufacturing (12%), these top three industries account for over 50% of all job postings, showing significant concentration in specific sectors.

At the other end of the spectrum, Retail Trade shows the lowest activity at just 3% of total postings, suggesting either market saturation or reduced hiring in traditional retail sectors. Manufacturing and Construction show moderate but steady demand at 12% and 8% respectively, indicating stable growth in these traditional sectors. This distribution reveals a clear shift towards knowledge-based and service-oriented industries, with traditional retail showing significantly lower activity compared to professional and technical services.

### Years of Experience by Industry
**Why this visualization?**
A box plot was selected to show the distribution of required years of experience across industries, revealing both the median requirements and any outliers.

**Key Insights:**
The analysis of experience requirements reveals significant variation across industries. Professional Services shows the widest range of requirements (0-15 years), indicating a diverse array of roles from entry-level to senior positions. Healthcare consistently requires higher minimum experience levels, with a median of 5 years, reflecting the specialized nature of the field. In contrast, Retail Trade has the lowest experience requirements, with a median of just 1 year.

Information Technology shows an interesting bimodal distribution in experience requirements, with peaks at 2 and 5 years, suggesting two distinct career paths within the sector. The Finance industry shows significant outliers, with some specialized roles requiring 10+ years of experience. Across all industries, the median experience requirement is 3 years, with 45% of postings requiring 3 or more years of experience. This distribution highlights the varying barriers to entry across different sectors and the importance of industry-specific experience requirements in job market dynamics.

### Remote vs. On-Site Jobs
**Why this visualization?**
A pie chart effectively shows the proportion of different work location types, giving a clear picture of remote work opportunities.

**Key Insights:**
The distribution of work arrangements shows a significant shift in workplace norms, with 35% of all job postings offering fully remote positions. Hybrid work arrangements account for 25% of postings, indicating a growing preference for flexible work models. However, traditional on-site positions still dominate at 40%, particularly in industries like Healthcare and Manufacturing.

The availability of remote work varies dramatically by industry. The Technology sector leads in remote work adoption, with 60% of positions offering remote options, while Healthcare maintains 80% on-site requirements. Remote work opportunities are primarily concentrated in Professional Services and IT sectors, while Manufacturing and Healthcare maintain predominantly on-site work arrangements. This distribution suggests a clear correlation between job type and remote work availability, with technical roles being three times more likely to offer remote options than customer-facing roles. These patterns reflect both the practical constraints of different industries and the evolving preferences in work arrangements post-pandemic.

# Conclusion

This analysis has provided valuable insights into the current job market through a comprehensive examination of the Lightcast job postings dataset. The data cleaning process successfully transformed the raw dataset into a clean, analysis-ready format by removing unnecessary columns, handling missing values, and eliminating duplicates. This rigorous cleaning process ensured the reliability of our subsequent analysis.

The exploratory data analysis revealed several key trends in the job market. The dominance of Professional, Scientific, and Technical Services (25% of postings) alongside Healthcare and Social Assistance (18%) indicates a strong demand for specialized knowledge workers and healthcare professionals. The analysis of experience requirements showed significant variation across industries, with Healthcare requiring the highest median experience (5 years) and Retail Trade the lowest (1 year). The remote work analysis revealed a significant shift in workplace norms, with 35% of positions offering fully remote options, though this varies dramatically by industry.

These findings have important implications for both job seekers and employers. Job seekers can use this information to target high-demand industries and understand the experience requirements for their desired roles. Employers can gain insights into industry standards for experience requirements and work arrangements. The clear industry-specific patterns in remote work availability also highlight the varying adaptability of different sectors to flexible work arrangements.

This analysis provides a solid foundation for further research into specific aspects of the job market, such as skill requirements, salary trends, or geographic distribution of opportunities. 


In [ ]:
#| label: enhanced-eda
#| echo: true
#| warning: false

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from wordcloud import WordCloud

# Set larger figure size
plt.rcParams["figure.figsize"] = (12, 6)

# --- Enhanced EDA ---

# 1. Top 10 Cities by Number of Job Postings
top_cities = df['CITY_NAME'].value_counts().nlargest(10)
fig = px.bar(
    x=top_cities.values,
    y=top_cities.index,
    orientation='h',
    labels={'x': 'Number of Postings', 'y': 'City'},
    title='Top 10 Cities by Number of Job Postings',
    width=800,
    height=500
)
fig.update_layout(yaxis=dict(autorange="reversed"))
fig.show()


# 2. Top 10 States by Number of Postings
top_states = df['STATE_NAME'].value_counts().nlargest(10)
fig = px.bar(
    x=top_states.index,
    y=top_states.values,
    labels={'x': 'State', 'y': 'Number of Postings'},
    title='Top 10 States by Number of Job Postings',
    width=900,
    height=500
)
fig.show()

# 3. Job Titles Word Cloud (if needed)
from wordcloud import WordCloud

text = ' '.join(df['TITLE_NAME'].dropna())
wordcloud = WordCloud(width=800, height=500, background_color='white').generate(text)

plt.figure(figsize=(10,5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Most Frequent Job Titles')
plt.show()

## Enhanced EDA: Analysis of Key Visualizations

### Top 10 Cities by Number of Job Postings
**Why this visualization?**  
A horizontal bar chart makes it easy to compare job demand across the top cities, especially with longer city names.

**Key Insights:**  
Job postings are heavily concentrated in major urban hubs like New York, Chicago, and Atlanta. These cities offer significantly more opportunities compared to others, suggesting that job seekers aiming for higher job availability should focus on these metropolitan areas.

---

### Top 10 States by Number of Job Postings
**Why this visualization?**  
A vertical bar chart effectively shows state-level hiring trends in an intuitive way.

**Key Insights:**  
Texas and California dominate in job postings, reflecting strong economies and large populations. Other states like Florida, Virginia, and Illinois also show high demand. After the top few, there's a noticeable decline, highlighting geographic concentration of job opportunities in a few states.

---

### Word Cloud of Most Frequent Job Titles
**Why this visualization?**  
A word cloud quickly identifies the most common job roles based on text frequency, providing a visual overview.

**Key Insights:**  
"Data Analyst" and "Consultant" emerge as the most prominent titles, emphasizing demand for roles in data, business analysis, and consulting. This suggests a job market leaning heavily toward analytical and strategic positions.

---

## Conclusion

The enhanced EDA highlights that job opportunities are geographically concentrated in certain states and cities, and technical and analytical roles dominate the job market. Candidates targeting these fields and locations can improve their employment prospects significantly.


---
title: Urban vs Rural States
jupyter: python3
---

In [ ]:
import pandas as pd

# Load cleaned dataset (update path)
df = pd.read_csv('region_analysis/lightcast_job_postings.csv')

# Convert SALARY to numeric and drop nulls
df['SALARY'] = pd.to_numeric(df['SALARY'], errors='coerce')
df = df.dropna(subset=['SALARY'])

# Tag AI vs Non-AI jobs
def tag_ai(row):
    title = str(row['TITLE_NAME']).lower()
    industry = str(row['NAICS_2022_2_NAME']).lower()
    
    if ('ai' in title or 'artificial intelligence' in title or 'machine learning' in title or
        'data scientist' in title or 'deep learning' in title or 'nlp' in title or 'ml' in title or
        'data engineer' in title or 'ml engineer' in title or 'scientist' in title or
        'computer vision' in title or 'robotics' in title or
        'professional, scientific, and technical services' in industry or 'information' in industry):
        return 'AI Career'
    else:
        return 'Non-AI Career'

df['Career_Type'] = df.apply(tag_ai, axis=1)
print(df['Career_Type'].value_counts())

# Group by STATE and Career Type → Avg Salary
state_salary = df.groupby(['STATE_NAME', 'Career_Type'])['SALARY'].mean().round(2).reset_index()
state_salary = state_salary.sort_values(by='SALARY', ascending=False)

# Show top 5 for each category
print("Top 5 States for AI Careers:")
print(state_salary[state_salary['Career_Type'] == 'AI Career'].head(5))

print("\nTop 5 States for Non-AI Careers:")
print(state_salary[state_salary['Career_Type'] == 'Non-AI Career'].head(5))

# Optional: save to CSV
# state_salary.to_csv('statewise_ai_vs_nonai_salary.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt

# Top 5 AI states
top_ai = state_salary[state_salary['Career_Type'] == 'AI Career'].head(5)

# Top 5 Non-AI states
top_nonai = state_salary[state_salary['Career_Type'] == 'Non-AI Career'].head(5)

# Plot AI Career Salaries
plt.figure(figsize=(8, 5))
plt.barh(top_ai['STATE_NAME'], top_ai['SALARY'])
plt.xlabel("Average Salary ($)")
plt.title("Top 5 States for AI Careers")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig("top_ai_states.png")
plt.show()

# Plot Non-AI Career Salaries
plt.figure(figsize=(8, 5))
plt.barh(top_nonai['STATE_NAME'], top_nonai['SALARY'], color='orange')
plt.xlabel("Average Salary ($)")
plt.title("Top 5 States for Non-AI Careers")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig("top_nonai_states.png")
plt.show()

In [ ]:
# Count of jobs used to compute avg salary
state_counts = df.groupby(['STATE_NAME', 'Career_Type'])['SALARY'].agg(['mean', 'count']).round(2).reset_index()
state_counts = state_counts.sort_values(by='mean', ascending=False)

# Show top 10 AI states by average salary + count
print(state_counts[state_counts['Career_Type'] == 'AI Career'].head(10))

In [ ]:
all_ai_states = state_counts[state_counts['Career_Type'] == 'AI Career']
print(all_ai_states['STATE_NAME'].unique())

In [ ]:
reliable_ai_states = state_counts[
    (state_counts['Career_Type'] == 'AI Career') &
    (state_counts['count'] >= 50)
].sort_values(by='mean', ascending=False)

print(reliable_ai_states.head(10))

In [ ]:
big_states = ['Massachusetts', 'New York', 'New Jersey', 'California', 'Texas']

ai_counts_big_states = df[
    (df['Career_Type'] == 'AI Career') & 
    (df['STATE_NAME'].isin(big_states))
].groupby('STATE_NAME')['SALARY'].agg(['count', 'mean']).round(2).sort_values(by='count', ascending=False)

ai_counts_big_states.rename(columns={'count': 'AI Job Postings', 'mean': 'Avg Salary ($)'}, inplace=True)

print(ai_counts_big_states)

In [ ]:
ai_median_salary = df[df['Career_Type'] == 'AI Career'].groupby('STATE_NAME')['SALARY'].median().round(2).reset_index()
ai_counts = df[df['Career_Type'] == 'AI Career'].groupby('STATE_NAME')['SALARY'].count().reset_index(name='Job_Count')

# Merge them
ai_summary = pd.merge(ai_median_salary, ai_counts, on='STATE_NAME')
ai_summary.columns = ['State', 'Median Salary ($)', 'AI Job Count']

# Optional: Filter for states with decent sample size
ai_summary = ai_summary[ai_summary['AI Job Count'] >= 50]

# Sort by Median Salary
ai_summary = ai_summary.sort_values(by='Median Salary ($)', ascending=False)

print(ai_summary.head(10))

In [ ]:
import matplotlib.pyplot as plt

top_states = ai_summary.head(10)

plt.figure(figsize=(10, 6))
plt.barh(top_states['State'], top_states['Median Salary ($)'], color='teal')
plt.xlabel("Median Salary ($)")
plt.title("Top 10 States for AI Careers (Median Salary, ≥50 Jobs)")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig("top_ai_states_median.png")
plt.show()

## Urban vs Rural States – Median AI Salary Comparison

This chart compares the median salaries of AI careers in selected **urban** (California, New York, Massachusetts) and **rural** (Arkansas, Montana, Nebraska) states. It highlights how urban hubs tend to offer significantly higher AI-related salaries, reinforcing the geographic wage gap in technology careers.

![](figures/urban_vs_rural_ai_salaries.png)


In [ ]:
# Heatmap
import seaborn as sns 
pivot = df.groupby(['STATE_NAME', 'Career_Type'])['SALARY'].median().unstack()
plt.figure(figsize=(14, 6))
sns.heatmap(pivot, annot=True, fmt=".0f", cmap="coolwarm", linewidths=0.5)
plt.title("Median Salary by State and Career Type")
plt.xlabel("Career Type")
plt.ylabel("State")
plt.tight_layout()
plt.show()

In [ ]:
# Group by state
ai_summary = df[df['Career_Type'] == 'AI Career'].groupby('STATE_NAME')['SALARY'].agg(
    Median_Salary='median', Job_Count='count', Std_Dev='std').reset_index()

plt.figure(figsize=(10, 6))
plt.scatter(ai_summary['Median_Salary'], ai_summary['Job_Count'],
            s=ai_summary['Job_Count'] / 1.5,
            c=ai_summary['Std_Dev'], cmap='viridis', alpha=0.7, edgecolors='black')
# Add state name labels to the bubbles
for i, row in ai_summary.iterrows():
    if row['Job_Count'] > 300:  # Label only big bubbles to reduce clutter
        plt.text(row['Median_Salary'], row['Job_Count'], row['STATE_NAME'],
                 fontsize=8, ha='center', va='center', color='black')

plt.colorbar(label='Salary Std Dev')
plt.xlabel('Median Salary ($)')
plt.ylabel('AI Job Count')
plt.title('AI Career Opportunities: Salary vs Availability by State')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
# Reference lines
median_salary_cutoff = ai_summary['Median_Salary'].median()
median_job_count_cutoff = ai_summary['Job_Count'].median()

plt.figure(figsize=(10, 6))
plt.scatter(ai_summary['Median_Salary'], ai_summary['Job_Count'],
            s=ai_summary['Job_Count'] / 1.5,
            c=ai_summary['Std_Dev'], cmap='viridis', alpha=0.7, edgecolors='black')

# Add quadrant lines
plt.axvline(median_salary_cutoff, color='gray', linestyle='--', linewidth=1)
plt.axhline(median_job_count_cutoff, color='gray', linestyle='--', linewidth=1)

# Label top states
for i, row in ai_summary.iterrows():
    if row['Job_Count'] > 300:
        plt.text(row['Median_Salary'], row['Job_Count'], row['STATE_NAME'],
                 fontsize=8, ha='center', va='center', color='black')

plt.colorbar(label='Salary Std Dev')
plt.xlabel('Median Salary ($)')
plt.ylabel('AI Job Count')
plt.title('AI Career Opportunities: Salary vs Availability (with Reference Lines)')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

### AI Salaries by State Heatmap


In [ ]:
import plotly.express as px

# Mapping of full state names to 2-letter codes
state_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
    'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH',
    'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC',
    'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA',
    'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN',
    'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY', 'District of Columbia': 'DC'
}

# Create dataframe with state abbreviations
ai_state_medians = df[df['Career_Type'] == 'AI Career'].groupby('STATE_NAME')['SALARY'].median().reset_index()
ai_state_medians['STATE_ABBR'] = ai_state_medians['STATE_NAME'].map(state_abbrev)

# Now plot
fig = px.choropleth(ai_state_medians,
                    locations='STATE_ABBR',
                    locationmode="USA-states",
                    color='SALARY',
                    scope="usa",
                    color_continuous_scale="Viridis",
                    labels={'SALARY':'Median AI Salary'})
fig.update_layout(title_text='Median AI Salary by State', geo=dict(showlakes=True))
fig.show()

In [ ]:
import plotly.graph_objects as go

# Prepare the base choropleth layer
fig = go.Figure(data=go.Choropleth(
    locations=ai_state_medians['STATE_ABBR'],  # 2-letter codes
    z=ai_state_medians['SALARY'],
    locationmode='USA-states',
    colorscale='Viridis',
    colorbar_title='Median AI Salary',
    text=ai_state_medians['STATE_NAME'],  # hover text
    hoverinfo='text+z'
))

# Add text annotations: state abbreviations
for i, row in ai_state_medians.iterrows():
    fig.add_trace(go.Scattergeo(
        locationmode='USA-states',
        lon=[None],  # plotly doesn’t support precise state centroids natively
        lat=[None],
        text=row['STATE_ABBR'],
        mode='text',
        textfont=dict(color='white', size=10),
        name=row['STATE_ABBR'],
        showlegend=False
    ))

# Update map layout
fig.update_layout(
    title_text='Median AI Salary by State (with State Labels)',
    geo=dict(
        scope='usa',
        showlakes=True,
        lakecolor='rgb(255, 255, 255)'
    )
)

fig.show()

#### Non AI Careers


In [ ]:
nonai_state_medians = df[df['Career_Type'] == 'Non-AI Career'].groupby('STATE_NAME')['SALARY'].median().reset_index()
nonai_state_medians['STATE_ABBR'] = nonai_state_medians['STATE_NAME'].map(state_abbrev)

In [ ]:
ai_top = ai_state_medians[['STATE_NAME', 'SALARY']].rename(columns={'SALARY': 'AI_Median'})
nonai_top = nonai_state_medians[['STATE_NAME', 'SALARY']].rename(columns={'SALARY': 'NonAI_Median'})

comparison = pd.merge(ai_top, nonai_top, on='STATE_NAME', how='inner')
comparison['Diff'] = comparison['AI_Median'] - comparison['NonAI_Median']
comparison.sort_values(by='Diff', ascending=False).head(10)

In [ ]:
# Step 1: Get AI job counts per state
ai_counts = df[df['Career_Type'] == 'AI Career'].groupby('STATE_NAME')['SALARY'].count().reset_index(name='AI_Job_Count')

# Step 2: Merge counts into comparison
comparison_with_counts = pd.merge(comparison, ai_counts, on='STATE_NAME', how='left')

# Step 3: Filter for states with at least 100 AI jobs
filtered = comparison_with_counts[comparison_with_counts['AI_Job_Count'] >= 300]

# Step 4: Take top 10 states by salary gap
top_realistic = filtered.sort_values(by='Diff', ascending=False).head(10).sort_values('AI_Median')

# Step 5: Plot dumbbell chart
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

# Draw connecting lines
for _, row in top_realistic.iterrows():
    plt.plot([row['NonAI_Median'], row['AI_Median']], [row['STATE_NAME']] * 2, color='gray', linewidth=2)

# Plot salary points
plt.scatter(top_realistic['NonAI_Median'], top_realistic['STATE_NAME'], color='darkorange', label='Traditional Median Salary', s=80)
plt.scatter(top_realistic['AI_Median'], top_realistic['STATE_NAME'], color='steelblue', label='AI Median Salary', s=80)

# Annotate the difference
for _, row in top_realistic.iterrows():
    plt.text(row['AI_Median'] + 1000, row['STATE_NAME'], f"+${int(row['Diff']):,}", fontsize=8, va='center', color='black')

plt.xlabel("Median Salary ($)")
plt.title("AI vs Traditional Salaries (Top 10 States, ≥300 AI Jobs)")
plt.legend()
plt.grid(axis='x', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

title: "Salaries Differ Across AI vs. Non-AI Careers"
subtitle: "Salary & Compensation Trends - Job Market Analysis 2024"
author:
  - name: An LY
    affiliations:
      - id: bu
        name: Boston University
        city: Boston
        state: MA
format: 
  html:
    toc: true
    number-sections: true
    df-print: paged
---



## Introduction

The rapid evolution of Artificial Intelligence (AI) has significantly reshaped the job market—creating high-demand, specialized roles while also influencing the trajectory of traditional careers. This project explores how salaries differ between AI-related and non-AI occupations using real-world job postings from the Lightcast dataset. Through classification, aggregation, and visualization, we aim to uncover compensation trends across both career paths.

## Dataset Overview

We analyzed a total of 72,498 U.S. job postings from the Lightcast database. Each role was tagged as either an AI Career or a Non-AI Career using keyword matching on the TITLE_NAME and NAICS_2022_2_NAME columns.

AI Careers: 28,310 postings(e.g., Data Scientist, Machine Learning Engineer, AI Engineer)

Non-AI Careers: 44,188 postings(e.g., Retail Sales, Administrative Assistant, Customer Service)

A custom rule-based function was used to assign each job into one of the two groups. We then filtered the dataset to include only postings with valid salary entries.


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import os
df = pd.read_csv('lightcast_job_postings.csv', low_memory=False)

job_counts = {'AI Careers': 28310, 'Non-AI Careers': 44188}

plt.figure(figsize=(8, 4))
plt.barh(list(job_counts.keys()), list(job_counts.values()), color=['steelblue', 'gray'])
plt.xlabel("Number of Job Postings")
plt.title("Distribution of AI vs. Non-AI Job Postings")
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.tight_layout()

plt.savefig("DATA/job_distribution_bar_chart.png", dpi=300, bbox_inches='tight')
plt.show()

## Salary Summary Table

The salary summary table below provides a statistical overview of compensation across AI and non-AI careers. This analysis is derived strictly from the Lightcast job postings dataset after filtering for valid salary entries.

Key metrics shown include mean, median, minimum, maximum, standard deviation, and the number of valid salary postings for each group. These statistics allow for a detailed comparison between the two career categories.


In [ ]:
import pandas as pd
import os
df = pd.read_csv('lightcast_job_postings.csv', low_memory=False)

df['SALARY'] = pd.to_numeric(df['SALARY'], errors='coerce')

def tag_ai(row):
    title = str(row['TITLE_NAME']).lower()
    industry = str(row['NAICS_2022_2_NAME']).lower()
    
    if ('ai' in title or 
        'artificial intelligence' in title or 
        'machine learning' in title or 
        'data scientist' in title or 
        'deep learning' in title or
        'nlp' in title or
        'computer vision' in title or
        'robotics' in title or
        'ml' in title or
        'data engineer' in title or
        'ml engineer' in title or
        'scientist' in title or
        ('professional, scientific, and technical services' in industry) or
        ('information' in industry)):
        return 'AI Career'
    else:
        return 'Non-AI Career'

df['Career_Type'] = df.apply(tag_ai, axis=1)

df_salary = df.dropna(subset=['SALARY'])

mean_salary_df = df_salary.groupby('Career_Type')['SALARY'].mean().round(2).reset_index()
mean_salary_df.columns = ['Career_Type', 'Mean_Salary ($)'] 

df['SALARY'] = pd.to_numeric(df['SALARY'], errors='coerce')
df_salary = df.dropna(subset=['SALARY'])

summary_stats = df_salary.groupby('Career_Type')['SALARY'].agg(
    count='count',
    mean='mean',
    median='median',
    min='min',
    max='max',
    std='std'
).round(2)

summary_stats = summary_stats.rename(columns={
    'count': 'Valid Salary Entries',
    'mean': 'Mean Salary ($)',
    'median': 'Median Salary ($)',
    'min': 'Minimum Salary ($)',
    'max': 'Maximum Salary ($)',
    'std': 'Standard Deviation ($)'
})

summary_stats['Job Postings Count'] = [28310 if i == 'AI Career' else 44188 for i in summary_stats.index]

summary_stats = summary_stats[['Job Postings Count', 'Valid Salary Entries', 'Mean Salary ($)',
                               'Median Salary ($)', 'Minimum Salary ($)', 'Maximum Salary ($)',
                               'Standard Deviation ($)']]
summary_stats

The table clearly shows that AI-related careers offer higher mean and median salaries compared to non-AI careers. The minimum salary for AI jobs is also notably higher, indicating a stronger salary floor. Although both groups have similar standard deviations, suggesting comparable salary variability, the overall compensation level for AI roles is substantially greater.

## Salary Distribution Visualization

The following boxplot visualizes the salary distributions for AI versus non-AI career categories. It graphically represents the median, interquartile ranges, and outliers for each group.


In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv('lightcast_job_postings.csv', low_memory=False)

# Tag careers before subsetting
def tag_ai(row):
    title = str(row['TITLE_NAME']).lower()
    industry = str(row['NAICS_2022_2_NAME']).lower()
    if ('ai' in title or 
        'artificial intelligence' in title or 
        'machine learning' in title or 
        'data scientist' in title or 
        'deep learning' in title or 
        'nlp' in title or 
        'computer vision' in title or 
        'robotics' in title or 
        'ml' in title or 
        'data engineer' in title or 
        'ml engineer' in title or 
        'scientist' in title or 
        'professional, scientific, and technical services' in industry or 
        'information' in industry):
        return 'AI Career'
    else:
        return 'Non-AI Career'

df['Career_Type'] = df.apply(tag_ai, axis=1)

# Now subset only rows with valid salary
df_salary = df.dropna(subset=['SALARY'])

# Plot without showing <Figure ...> output
ax = df_salary.boxplot(column='SALARY', by='Career_Type', figsize=(10,6))
plt.title('Salary Comparison: AI vs Non-AI Careers')
plt.suptitle('')
plt.xlabel('Career Type')
plt.ylabel('Salary ($)')
plt.grid(True)

plot_path = os.path.join('DATA', 'ai_vs_nonai_salary_comparison.png')
plt.savefig(plot_path, dpi=300, bbox_inches='tight')
plt.show()

The boxplot highlights that AI careers have a higher median salary and a more compressed lower salary range compared to non-AI careers. While both groups exhibit high-end salary outliers (up to $500,000), non-AI careers show a greater spread of lower-end salaries. This suggests that AI careers offer more consistent and stable compensation, whereas non-AI jobs have more variability, particularly toward lower-paying positions.

## Forecasting AI Career Trends 
To explore future patterns, we used a simple linear regression model to forecast average AI salaries and job posting volumes over the next six months.


In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import statsmodels.api as sm


df = pd.read_csv('lightcast_job_postings.csv', low_memory=False)

df['POSTED'] = pd.to_datetime(df['POSTED'], errors='coerce')
df['SALARY'] = pd.to_numeric(df['SALARY'], errors='coerce')

def tag_ai(row):
    title = str(row['TITLE_NAME']).lower()
    industry = str(row['NAICS_2022_2_NAME']).lower()
    if ('ai' in title or 'artificial intelligence' in title or 'machine learning' in title or
        'data scientist' in title or 'deep learning' in title or 'nlp' in title or
        'computer vision' in title or 'robotics' in title or 'ml' in title or
        'data engineer' in title or 'ml engineer' in title or 'scientist' in title or
        'professional, scientific, and technical services' in industry or 'information' in industry):
        return 'AI Career'
    else:
        return 'Non-AI Career'

df['Career_Type'] = df.apply(tag_ai, axis=1)

df_valid = df.dropna(subset=['POSTED'])
monthly_counts = df_valid.groupby([df_valid['POSTED'].dt.to_period('M'), 'Career_Type']) \
                         .size().unstack().fillna(0)
monthly_counts.index = monthly_counts.index.to_timestamp()

plt.figure(figsize=(10, 5))
monthly_counts.plot(kind='line', marker='o', ax=plt.gca())
plt.title("Monthly Job Postings: AI vs. Non-AI Careers")
plt.ylabel("Number of Postings")
plt.xlabel("Month")
plt.grid(True)
plt.tight_layout()
job_postings_path = "DATA/monthly_job_postings_forecast.png"
plt.savefig(job_postings_path, dpi=300)
plt.show()

df_ai = df[(df['Career_Type'] == 'AI Career') & (~df['SALARY'].isna())]
monthly_salary = df_ai.groupby(df_ai['POSTED'].dt.to_period('M'))['SALARY'].mean().dropna()
monthly_salary.index = monthly_salary.index.to_timestamp()

X = sm.add_constant(range(len(monthly_salary)))
y = monthly_salary.values
model = sm.OLS(y, X).fit()

future_periods = 6
future_X = sm.add_constant(range(len(monthly_salary), len(monthly_salary) + future_periods))
forecast = model.predict(future_X)

last_date = monthly_salary.index[-1]
future_dates = [last_date + pd.DateOffset(months=i+1) for i in range(future_periods)]

plt.figure(figsize=(10, 5))
plt.plot(monthly_salary.index, monthly_salary.values, marker='o', label='Historical Avg Salary (AI)')
plt.plot(future_dates, forecast, marker='x', linestyle='--', label='Forecast (Next 6 Months)')
plt.title("Forecast: Average AI Salary Over Time")
plt.ylabel("Salary ($)")
plt.xlabel("Month")
plt.grid(True)
plt.legend()
plt.tight_layout()
salary_forecast_path = "DATA/ai_salary_forecast.png"
plt.savefig(salary_forecast_path, dpi=300)
plt.show()

The analysis of monthly job postings reveals that Non-AI careers consistently dominate the U.S. job market, averaging around 9,000 postings per month. Despite experiencing a noticeable dip in July, Non-AI job volumes quickly rebounded in August and September, indicating strong demand recovery. In contrast, AI careers maintain a lower volume, typically ranging between 5,000 and 6,000 postings monthly. However, the trend among AI roles appears more stable, with minor fluctuations and a modest recovery following July’s decline. This suggests that while fewer in quantity, AI job postings demonstrate resilience and steady momentum in hiring activity.

Looking at the salary trends for AI careers, historical data from May to September 2024 shows moderate fluctuations, with a pronounced dip in August representing the lowest average salary in that period. Using a simple linear regression model, forecasts from October 2024 through March 2025 indicate a gradual decline in average AI salaries. Importantly, this downward trend does not suggest collapse but rather points to a stabilizing market. As AI technologies become more widespread and accessible, the normalization of compensation—paired with a growing supply of skilled candidates—could be moderating previously inflated salary levels. Despite the projected dip, AI roles remain high-paying and continue to offer strong long-term potential.



## Key Insights

AI-related careers consistently offer higher salaries, with a mean of $133,344 and median of $130,500, compared to $108,513 and $102,500 respectively for non-AI roles.

Salary variability remains relatively equal across both groups (~$43,000 standard deviation), but AI roles tend to avoid the lower-end salaries more common in non-AI positions.

The minimum salary for AI careers is substantially higher at $23,585, indicating a stronger baseline earning potential versus $15,860 for non-AI roles.

The maximum reported salary is identical across both categories ($500,000), likely reflecting high-level executive or specialized niche roles.

AI job postings, while fewer in number, show consistent volume and a stable rebound after a mid-year dip, suggesting continued demand for AI talent.

Forecasting results predict a gradual decline in AI average salaries over the next six months, indicating potential market stabilization rather than contraction.

## Conclusion

This analysis confirms that AI-related roles provide superior salary outcomes, both in average and median terms, and offer greater compensation stability at the lower end of the spectrum. Despite a projected slight decline in average salaries, AI positions remain highly competitive and rewarding.

These findings emphasize the value of developing AI-focused skill sets such as machine learning, NLP, and data engineering. For students, job seekers, and educators alike, aligning educational and career strategies with the AI sector’s continued evolution will be key to unlocking long-term financial and professional success.


---
title: "Exploratory Data Analysis"
subtitle: "Uncovering Patterns in the Job Market Dataset"
author:
  - name: Group 10
    affiliations:
      - id: bu
        name: Boston University
        city: Boston
        state: MA
format: 
  html:
    toc: true
    number-sections: true
    df-print: paged
---



# Introduction

This EDA explores key characteristics of the job postings dataset, including salary distribution, industry representation, geographic trends, and temporal patterns. The analysis provides insights that guide downstream tasks such as skill gap analysis, clustering, and prediction.

# Dataset Overview


In [ ]:
import pandas as pd

df = pd.read_csv('region_analysis/lightcast_job_postings.csv')
df.info()
df.head()

# Handling Missing Data


In [ ]:
df.isnull().mean().sort_values(ascending=False).head(10)

# Salary Distribution


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

df['SALARY'] = pd.to_numeric(df['SALARY'], errors='coerce')
plt.figure(figsize=(10,6))
sns.histplot(df['SALARY'], bins=50, kde=True)
plt.title("Salary Distribution")
plt.xlabel("Salary ($)")
plt.ylabel("Frequency")
plt.xlim(0, 300000)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Top 10 Job Titles


In [ ]:
df['TITLE_NAME'].value_counts().head(10)

# Top 10 Industries


In [ ]:
df['NAICS_2022_2_NAME'].value_counts().head(10)

# Job Postings by State


In [ ]:
state_counts = df['STATE_NAME'].value_counts().head(15)
state_counts.plot(kind='barh', figsize=(10,6), color='steelblue')
plt.title("Top 15 States by Job Postings")
plt.xlabel("Number of Postings")
plt.gca().invert_yaxis()
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

# Posting Trends Over Time


In [ ]:
df['POSTED'] = pd.to_datetime(df['POSTED'], errors='coerce')
df['POSTED'].dt.to_period('M').value_counts().sort_index().plot(kind='line', figsize=(12,6))
plt.title("Job Postings Over Time")
plt.ylabel("Number of Postings")
plt.xlabel("Month")
plt.grid(True, alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
print(df.columns.tolist())

title: "Skill Gap Analysis"
subtitle: "Comprehensive Data Cleaning & Exploratory Analysis of Job Market Trends"
author:
  - name: "Ritusri Mohan"
    affiliations:
      - id: bu
        name: Boston University
        city: Boston
        state: MA
format: 
  html:
    toc: true
    number-sections: true
    df-print: paged
---



# Skill Gap Analysis



In this analysis, we evaluate the current skill levels of our team and compare them with industry requirements based on job postings data. This will help identify skill gaps and propose improvement strategies.



## Team Skill Level Data


In [ ]:
#| label: clean-data
#| echo: true
#| warning: false

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Load dataset
csv_path = 'region_analysis/lightcast_job_postings.csv'
df = pd.read_csv(csv_path)

# Drop unnecessary columns
columns_to_drop = [
    "ID", "URL", "ACTIVE_URLS", "DUPLICATES", "LAST_UPDATED_TIMESTAMP",
    "NAICS2", "NAICS3", "NAICS4", "NAICS5", "NAICS6",
    "SOC_2", "SOC_3", "SOC_5"
]
df.drop(columns=columns_to_drop, inplace=True)

# Drop columns with >50% missing
df.dropna(thresh=len(df) * 0.5, axis=1, inplace=True)

# Fill missing values
for col in df.select_dtypes(include=[np.number]):
    df[col].fillna(df[col].median(), inplace=True)

for col in df.select_dtypes(include=['object']):
    df[col].fillna("Unknown", inplace=True)

# Drop duplicates
df.drop_duplicates(subset=["TITLE", "COMPANY", "LOCATION", "POSTED"], keep="first", inplace=True)

## Top 5 and Top 8 Software Skills from Job Postings


In [ ]:
#| label: top-software-skills
#| echo: true

# Get top software skills from job postings
software_skills = (
    df['SOFTWARE_SKILLS_NAME']
    .dropna()
    .str.replace(r'[\[\]\n]', '', regex=True)  # Remove brackets and newlines
    .str.split(',')
    .explode()
    .str.strip()
)

# Filter out empty strings
software_skills = software_skills[software_skills != '']

# Get top 5 most frequent software skills
top_5_software_skills = software_skills.value_counts().head(5)

# Extract skill names
top_5_skills = top_5_software_skills.index.tolist()

print("Top 5 software skills from job postings:")
print(top_5_software_skills)

# Get top 8 most frequent software skills
top_8_software_skills = software_skills.value_counts().head(8)

# Extract skill names
top_8_skills = top_8_software_skills.index.tolist()

print("Top 8 software skills from job postings:")
print(top_8_software_skills)

In [ ]:
#| label: team-skills-5
#| echo: true

import pandas as pd

# Define team skill levels (Proficiency scale: 1=Beginner, 5=Expert)
skills_data = {
    "Name": ["Shreya", "An Ly", "Advait", "Ritusri"],
    "SQL (Programming Language)": [3, 2, 2, 3],
    "Microsoft Excel": [4, 3, 3, 4],
    "Python (Programming Language)": [3, 3, 3, 3],
    "SAP Applications": [2, 1, 3, 2],
    "Dashboard": [2, 2, 3, 4]
}

df_skills = pd.DataFrame(skills_data)
df_skills.set_index("Name", inplace=True)
df_skills


In [ ]:
#| label: heatmap-top5
#| echo: true

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Team skills for top 5
skills_data_top5 = {
    "Name": ["Shreya", "An Ly", "Advait", "Ritusri"],
    "SQL (Programming Language)": [3, 2, 2, 3],
    "Microsoft Excel": [4, 3, 3, 4],
    "Python (Programming Language)": [3, 3, 3, 3],
    "SAP Applications": [2, 1, 3, 2],
    "Dashboard": [2, 2, 3, 4]
}

df_skills_top5 = pd.DataFrame(skills_data_top5).set_index("Name")

# Plot Heatmap 1
plt.figure(figsize=(10, 5))
sns.heatmap(df_skills_top5, annot=True, cmap="YlOrBr", linewidths=0.5)
plt.title("Team Skill Levels – Top 5 Software Skills")
plt.show()

In [ ]:
#| label: team-skills-8
#| echo: true

import pandas as pd

# Define team skill levels (Proficiency scale: 1=Beginner, 5=Expert)
skills_data1 = {
    "Name": ["Shreya", "An Ly", "Advait", "Ritusri"],
    "SQL (Programming Language)": [3, 2, 2, 3],
    "Microsoft Excel": [4, 3, 3, 4],
    "Python (Programming Language)": [3, 3, 3, 3],
    "SAP Applications": [2, 1, 3, 2],
    "Dashboard": [2, 2, 3, 4],
    "Tableau (Business Intelligence Software)":[0, 0, 0, 0],
    "Power BI":[0, 0, 0, 0],
    "Microsoft Office":[0, 0, 0, 0],
    
}

df_skills1 = pd.DataFrame(skills_data1)
df_skills1.set_index("Name", inplace=True)
df_skills1

In [ ]:
#| label: heatmap-top8
#| echo: true

import pandas as pd

# Define team skill levels (Proficiency scale: 1=Beginner, 5=Expert)
skills_data1 = {
    "Name": ["Shreya", "An Ly", "Advait", "Ritusri"],
    "SQL (Programming Language)": [3, 2, 2, 3],
    "Microsoft Excel": [4, 3, 3, 4],
    "Python (Programming Language)": [3, 3, 3, 3],
    "SAP Applications": [2, 1, 3, 2],
    "Dashboard": [2, 2, 3, 4],
    "Tableau (Business Intelligence Software)":[0, 0, 0, 0],
    "Power BI":[0, 0, 0, 0],
    "Microsoft Office":[0, 0, 0, 0],
    
}
df_skills_top8 = pd.DataFrame(skills_data1).set_index("Name")

# Plot Heatmap 2
plt.figure(figsize=(10, 5))
sns.heatmap(df_skills_top8, annot=True, cmap="YlOrBr", linewidths=0.5)
plt.title("Team Skill Levels – Top 8 Software Skills")
plt.show()

### Improvement Plan
The heatmap analysis indicates that while the team has solid grounding in fundamental IT tools, there are significant gaps in certain enterprise and visualization technologies that are highly demanded in job postings. Addressing these gaps will elevate the team’s capabilities and make them more market-ready.

SQL (Programming Language)
Although the team has intermediate proficiency in SQL, moving towards expert-level understanding is essential. SQL remains one of the most requested skills across IT and data-centric roles. Advanced topics like complex joins, window functions, performance tuning, and writing optimized queries should be covered. Recommended resources include Coursera's SQL for Data Science and DataCamp SQL Career Track.

Python (Programming Language)
Python is the cornerstone of automation, analytics, and machine learning. The team should focus on advancing their skills beyond basic scripting. This includes learning about data manipulation (Pandas), building small projects, and exploring libraries used in data science or automation. Free resources like the Google Python Crash Course and the IBM Python for Data Science (Coursera) can be leveraged.

SAP Applications
SAP is currently one of the weakest areas within the team. Enterprise software skills like SAP are critical in corporate environments. Team members should target introductory SAP courses to gain a basic understanding of navigation, reporting, and common SAP modules. Free options like OpenSAP are recommended to start with.

Dashboarding Tools (Power BI / Tableau)
Visual storytelling and data presentation are crucial skills in modern IT roles. As seen in the analysis, there is little to no current exposure to Power BI and Tableau. Training should start with beginner-friendly courses that teach how to create interactive dashboards and perform basic data analysis. Tableau Public and Microsoft Learn for Power BI offer excellent free resources.

Microsoft Excel
While the team is relatively advanced in Excel, moving towards expert proficiency will unlock greater efficiencies. Topics like VBA macros, Power Query, pivot tables, and advanced formulas should be prioritized. Recommended platforms include LinkedIn Learning and Excel Skills for Business (Coursera).

Tableau, Power BI, and Microsoft Office (Missing Skills)
The missing ratings for Tableau, Power BI, and Microsoft Office indicate complete lack of knowledge. These tools are essential for reporting, business communication, and project collaboration in IT environments. Each team member should aim to reach at least intermediate level proficiency through self-paced courses and hands-on practice.

#### Suggested Team Learning Plan
To close these gaps systematically:

Weekly Learning Sessions: Rotate weekly responsibilities where each member shares what they learned about a specific skill.

Pair Learning: Pair up team members with stronger skills (e.g., SQL, Python) to support others.

Micro-Certifications: Encourage completion of short certifications within 4–6 weeks (e.g., Tableau Desktop Specialist, Power BI Fundamentals).

Practical Projects: Apply new skills through internal mock projects like building dashboards or writing Python scripts.

Tracking Progress: Regularly assess skill levels every month and update the heatmap to measure progress.

### Conclusion
The skill gap analysis clearly highlights the team’s current strengths and opportunities for improvement. While foundational skills like SQL, Python, and Excel are relatively well developed, enterprise tools like SAP, Tableau, and Power BI remain significant blind spots.

By following the improvement plan and encouraging structured learning, the team will not only close existing gaps but also align with modern IT and data industry expectations. This proactive approach will make the team more versatile, industry-relevant, and ready for advanced career opportunities.

---


---
title: "ML Methods"
subtitle: "Predicting Job Posting Duration Using Random Forest Regressor"
author:
  - name: "Shreya Mani"
    affiliations:
          - id: bu
            name: Boston University
            city: Boston
            state: MA
format: 
  html:
        toc: true
        number-sections: true
        df-print: paged
---


# Introduction

In this machine learning project, I aimed to predict how long job postings remain active (i.e., their DURATION) using a Random Forest Regressor. The dataset contains job postings with features such as minimum years of experience, employment type, remote work status, internship status, and required education levels. My goal was to build a predictive model, evaluate its performance using the Mean Squared Error (MSE), and visualize the results with a scatter plot comparing actual and predicted durations. This analysis can help organizations understand factors influencing job posting durations, aiding in recruitment planning.

# Data Preprocessing

I started by loading the dataset and selecting a subset of features relevant to predicting DURATION. The features I chose were MIN_YEARS_EXPERIENCE, EMPLOYMENT_TYPE, REMOTE_TYPE, IS_INTERNSHIP, and EDUCATION_LEVELS, as they likely influence how long a job posting stays active. I handled missing values in the target variable (DURATION) by dropping rows with missing data.

A challenge arose with the EDUCATION_LEVELS column, which contained string representations of lists . To address this, I wrote a preprocessing function to parse these strings, extract the first numerical value from each list, and convert it to an integer. This ensured that all features were numerical, as required by the Random Forest Regressor. The dataset was then split into training (80%) and testing (20%) sets to evaluate the model performance on unseen data.

Here the Python code I used for data preprocessing:


In [ ]:
#| label: ml data cleaning
#| echo: true
#| warning: false

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import ast
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Auto-download CSV if missing
csv_path = 'region_analysis/lightcast_job_postings.csv'
if not os.path.exists(csv_path):
    print(f"{csv_path} not found! Attempting to download...")
    os.makedirs('region_analysis', exist_ok=True)
    try:
        import gdown
    except ImportError:
        print("Installing gdown...")
        !pip install gdown
        import gdown
    file_id = '1V2GCHGt2dkFGqVBeoUFckU4IhUgk4ocQ'  # Replace with actual file ID
    url = f'https://drive.google.com/uc?id={file_id}'
    try:
        gdown.download(url, csv_path, quiet=False)
        print("Download complete!")
    except Exception as e:
        print(f"Download failed: {e}")
        raise
else:
    print(f"{csv_path} found. Proceeding...")

# Load the dataset
try:
    df = pd.read_csv(csv_path)
    print(f"Initial dataset size: {df.shape}")
    print(f"Missing values:\n{df[['DURATION', 'MIN_YEARS_EXPERIENCE', 'EMPLOYMENT_TYPE', 'REMOTE_TYPE', 'MIN_EDULEVELS']].isnull().sum()}")
except Exception as e:
    print(f"Error loading CSV: {e}")
    raise

## Sample of preprocessed EDUCATION_LEVELS


In [ ]:
# Define function to parse MIN_EDULEVELS strings
def parse_education_levels(edu):
    if isinstance(edu, (int, float)) and not np.isnan(edu):
        return int(edu)  # Return integer if already numerical
    if isinstance(edu, str):
        try:
            edu_list = ast.literal_eval(edu.replace('\n', ''))
            return int(edu_list[0]) if isinstance(edu_list[0], (int, float)) else np.nan
        except (ValueError, SyntaxError, IndexError) as e:
            print(f"Parsing failed for: {edu}, Error: {e}")
            return np.nan
    return np.nan

# Select features and target
features = ['MIN_YEARS_EXPERIENCE', 'EMPLOYMENT_TYPE', 'REMOTE_TYPE', 'IS_INTERNSHIP', 'MIN_EDULEVELS']
target = 'DURATION'

# Check if all features and target exist
missing_cols = [col for col in features + [target] if col not in df.columns]
if missing_cols:
    print(f"Missing columns: {missing_cols}")
    # If IS_INTERNSHIP is missing, remove it from features
    if 'IS_INTERNSHIP' in missing_cols:
        features.remove('IS_INTERNSHIP')
    else:
        raise ValueError("Required columns not found in dataset")

# Create a copy of the dataset with selected columns
df_subset = df[features + [target]].copy()

# Parse MIN_EDULEVELS
df_subset['MIN_EDULEVELS'] = df_subset['MIN_EDULEVELS'].apply(parse_education_levels)

# Handle missing values with imputation for all numerical columns
num_cols = [col for col in features if col in df_subset.columns]
num_imputer = SimpleImputer(strategy='median')
df_subset[num_cols] = num_imputer.fit_transform(df_subset[num_cols])

# Impute DURATION with mean
df_subset['DURATION'] = df_subset['DURATION'].fillna(df_subset['DURATION'].mean())

# Ensure IS_INTERNSHIP is integer if present
if 'IS_INTERNSHIP' in df_subset.columns:
    df_subset['IS_INTERNSHIP'] = df_subset['IS_INTERNSHIP'].astype(int)

# Verify no missing values
print(f"Missing values after imputation:\n{df_subset.isnull().sum()}")
print(f"Preprocessed dataset size: {df_subset.shape}")

# Features and target
X = df_subset[num_cols]
y = df_subset['DURATION']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set size:", X_train.shape)
print("Testing set size:", X_test.shape)
print("Sample of preprocessed MIN_EDULEVELS:", df_subset['MIN_EDULEVELS'].head().tolist())

# Model Training

With the data preprocessed, I trained a Random Forest Regressor, a robust model that handles numerical features well and is less prone to overfitting. The model was trained on the training set with 100 trees (n_estimators=100) to ensure stable predictions. Random Forest works by building multiple decision trees and averaging their predictions, which often leads to better performance compared to a single decision tree.

Here is the code for training the Random Forest Regressor:


In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Initialize and train the Random Forest Regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

print("Model training completed.")

# Model Evaluation and Visualization
After training the model, I used it to predict the DURATION for the test set. To evaluate the model performance, I calculated the Mean Squared Error (MSE), which measures the average squared difference between actual and predicted values. A lower MSE indicates better predictive accuracy.

I also created a scatter plot to visualize the model performance, comparing the actual DURATION values to the predicted ones. A red dashed line represents perfect predictions (where actual equals predicted). Points closer to this line indicate better predictions.

Here is the code for evaluation and visualization:


In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

# Cross-validation
cv_scores = cross_val_score(rf, X, y, cv=5, scoring='neg_mean_squared_error')
cv_mse = -cv_scores.mean()
cv_rmse = np.sqrt(cv_mse)
print(f"Cross-validated MSE: {cv_mse:.2f} ± {cv_scores.std():.2f}")
print(f"Cross-validated RMSE: {cv_rmse:.2f} days")

# Predict on test set
y_pred = rf.predict(X_test)

# Calculate MSE and RMSE on test set
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Test set MSE: {mse:.2f}")
print(f"Test set RMSE: {rmse:.2f} days")

# Feature importance
feature_importance = pd.DataFrame({
    'Feature': num_cols,
    'Importance': rf.feature_importances_
}).sort_values('Importance', ascending=False)
print("\nFeature Importance:")
print(feature_importance)

# Plot actual vs predicted values
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, color='blue', alpha=0.5, label='Predicted vs Actual')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label='Perfect Prediction')
plt.xlabel('Actual Duration (Days)')
plt.ylabel('Predicted Duration (Days)')
plt.title('Random Forest Regressor: Actual vs Predicted Job Posting Duration')
plt.legend()
plt.grid(True)
plt.savefig('actual_vs_predicted_duration.png')
plt.show()

title: "NLP Methods"
subtitle: "NLP Analysis: Extracting Required Skills from Job Postings"
author:
  - name: Shreya Mani
    affiliations:
      - id: bu
        name: Boston University
        city: Boston
        state: MA
format: 
  html:
        toc: true
        number-sections: true
        df-print: paged
---



# Introduction
In this project, we used Natural Language Processing (NLP) to extract required skills from job postings based on their description text in the BODY column. The dataset contains job postings with unstructured text descriptions, which often mention skills needed for the role (e.g., "analyze data" or "develop software"). Our goal was to identify and analyze the most common skills mentioned in these postings, providing insights into the skills most in demand. This analysis can help job seekers understand the key skills to develop and assist employers in identifying trends in skill requirements.

# Data Preprocessing
We started by loading the dataset and focusing on the BODY column, which contains the job description text. The BODY text is unstructured and requires preprocessing for NLP analysis. We performed the following steps:

Tokenization and Cleaning: Converted the text to lowercase, removed punctuation, and tokenized the text into words.


Stop Word Removal: Removed common stop words (e.g., "the", "is") that don’t add meaningful information. We used a predefined list of common stop words to avoid external dependencies.


Skill Extraction: Defined a list of common skills relevant to job postings (e.g., "data analysis," "software development") and searched for these skills in the cleaned text. For simplicity, we used keyword matching to identify skills, but this could be extended with more advanced NLP techniques like named entity recognition (NER) or pre-trained models.

Since this task is exploratory and doesn’t require a target variable, we didn’t split the data into training and testing sets. Instead, we processed all available job descriptions to extract and analyze skills.

Here’s the Python code we used for data preprocessing and skill extraction:


In [ ]:
#| label: skill-extraction
#| echo: true
#| warning: false

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
from collections import Counter

# Load dataset
csv_path = 'region_analysis/lightcast_job_postings.csv'
df = pd.read_csv(csv_path)

# Drop rows with missing job descriptions
df = df.dropna(subset=['BODY'])

# Define stop words
stop_words = {
    'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he',
    'in', 'is', 'it', 'its', 'of', 'on', 'that', 'the', 'to', 'was', 'were', 'will',
    'with', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your',
    'yours', 'yourself', 'yourselves', 'him', 'his', 'her', 'hers', 'herself', 'it',
    'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which',
    'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were',
    'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing',
    'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of',
    'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during',
    'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on',
    'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when',
    'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other',
    'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very',
    's', 't', 'can', 'will', 'just', 'don', 'should', 'now'
}

# Clean text
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = text.split()
    return ' '.join([word for word in tokens if word not in stop_words])

df['BODY_CLEANED'] = df['BODY'].apply(clean_text)

# Define common skills
skills_list = [
    "data analysis", "software development", "machine learning",
    "project management", "communication", "teamwork",
    "sql", "python", "modeling", "analytics"
]

# Extract skills
def extract_skills(text):
    found_skills = []
    for skill in skills_list:
        if skill in text:
            found_skills.append(skill)
    return found_skills

df['SKILLS'] = df['BODY_CLEANED'].apply(extract_skills)

# Flatten skill list and count
all_skills = [skill for sublist in df['SKILLS'] for skill in sublist]
skill_counts = Counter(all_skills)



# Visualize top skills
plt.figure(figsize=(10, 6))
if skill_counts:
    skills, counts = zip(*sorted(skill_counts.items(), key=lambda x: x[1], reverse=True))
    plt.bar(skills, counts, color='skyblue')
    plt.xlabel('Skills')
    plt.ylabel('Frequency')
    plt.title('Most Common Skills in Job Postings')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig('skills_frequency.png')
    plt.show()
else:
    print("No skills were found in the job descriptions.")

# Skill Analysis and Visualization

After extracting skills from the job descriptions, we analyzed their frequency to identify the most common skills mentioned. We visualized the results using a bar plot, showing the count of each skill across all job postings. This helps highlight the skills that are most in demand based on the dataset.

Here’s the code we used for analyzing and visualizing the skills:


title: "References"
---



## Predictive Salary Modelling: Leveraging Data Science Skills and Machine Learning for Accurate Forecasting
Haseeb, M. A., Viswanathan, R., Iyer, K., Hota, A. R., & Prathaban, B. P. (2024). Predictive salary modelling: Leveraging data science skills and machine learning for accurate forecasting. 2024 9th International Conference on Communication and Electronics Systems (ICCES), 1011–1019. https://ieeexplore.ieee.org/document/10859447

## Tackling Economic Inequalities Through Business Analytics: A Literature Review
Adaga, E. M., Egieya, Z. E., Ewuga, S. K., Abdul, A. A., & Abrahams, O. (2024). Tackling economic inequalities through business analytics: A literature review. Computer Science & IT Research Journal, 5(1), 60–80. https://doi.org/10.51594/csitjr.v5i1.702

## Antecedent Configurations Toward Supply Chain Resilience: The Joint Impact of Supply Chain Integration and Big Data Analytics Capability
Jiang, Y., Feng, T., & Huang, Y. (2024). Antecedent configurations toward supply chain resilience: The joint impact of supply chain integration and big data analytics capability. Journal of Operations Management, 70(2), 257–284. https://doi.org/10.1002/joom.1282

## Leveraging AI and Data Analytics for Enhancing Financial Inclusion in Developing Economies
Adeoye, O. B., Addy, W. A., Ajayi-Nifise, A. O., Odeyemi, O., Okoye, C. C., & Ofodile, O. C. (n.d.). Leveraging AI and data analytics for enhancing financial inclusion in developing economies. Finance & Accounting Research Journal. https://fepbl.com/index.php/farj/article/view/856

## Employee Career Decision Making: The Influence of Salary and Benefits, Work Environment and Job Security
Achim, N., Badrolhisam, N. I., & Zulkipli, N. (2019). Employee career decision making: The influence of salary and benefits, work environment and job security. Journal of Academia, 7(Special Issue 1), 41–50.

## The Influence of Salaries and “Opportunity Costs” on Teachers’ Career Choices
Murnane, R. J., Singer, J. D., & Willett, J. B. (1989). The influences of salaries and “opportunity costs” on teachers' career choices: Evidence from North Carolina. Harvard Educational Review, 59(3), 325–349.

## The Future of Work: Impacts of AI on Employment and Job Market Dynamics
Tomar, A., Sharma, S., Arti, & Suman, S. (2024). The future of work: Impacts of AI on employment and job market dynamics. 2024 International Conference on Progressive Innovations in Intelligent Systems and Data Science (ICPIDS).

## AI and Job Market: Analysing the Potential Impact of AI on Employment, Skills, and Job Displacement
Faluyi, S. E. (2025). AI and job market: Analysing the potential impact of AI on employment, skills, and job displacement. African Journal of Marketing Management, 17(1), 1–8.
